## Clasificación de texto

Para entender el efecto de la cantidad de dimensiones en la capacidad predictiva del modelo vamos a trabajar en un clasificador de noticias de dos diarios argentinos para intentar distinguir entre uno y otro a partir del vocabulario que utilizan. <br />

Para esto vamos a implementar un modelo de tipo Naïve Bayes con vectorización de tipo TF-IDF.

In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score,classification_report,auc,roc_curve,roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
seed=42
import unicodedata
from nltk.corpus import stopwords
stop = stopwords.words('spanish')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize as TK
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

# Para visualizar
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_clarin = pd.read_csv('clarin.csv')
df_clarin['class'] = 0

In [3]:
df_p12 = pd.read_csv('pagina12.csv')
df_p12['class'] = 1

In [4]:
df = pd.concat([df_clarin,df_p12])

In [5]:
df.sample(10)

,Unnamed: 0,cuerpo,fecha_hora,imagen,resumen,suplemento,titulo,url,class
35,35,Una ordenanza municipal de Córdoba que orden...,10/10/2017,//images.clarin.com/2017/10/10/rJhLc_qnb_930x5...,La polémica se generó por una ordenanza que im...,/sociedad/,La guerra del azúcar: Tucumán va a la Justicia...,http://www.clarin.com/sociedad/guerra-azucar-t...,0
53,53,Netflix y Marvel decidieron cancelar la presen...,09 de octubre de 2017,https://images.pagina12.com.ar/styles/focal_16...,NaN,NaN,Programados,https://www.pagina12.com.ar/67937-programados,1
252,252,Era un día más en la agitada rutina de Magui...,12/09/2017,//images.clarin.com/2017/09/12/Bkpk318qb_930x5...,Le rompieron el vidrio de su auto para sacarle...,/deportes/hockey/,Susto para una ex Leona: Magui Aicega fue asal...,http://www.clarin.com/deportes/hockey/susto-ex...,0
312,312,"La actriz Karina K, talentosa artista del mu...",09/10/2017,//images.clarin.com/2015/07/08/SkZde_3RQg_930x...,"De gira con su espectáculo, quedó varada en Pu...",/espectaculos/teatro/,La denuncia de Karina K: la actriz dice haber ...,http://www.clarin.com/espectaculos/teatro/denu...,0
6,6,Un clásico del deporte nacional empezó en su...,10/10/2017,//images.clarin.com/2017/10/10/r141i9q3W_930x5...,El cordobés no faltó a la acción deportiva en ...,/lo-ultimo/,José Meolans estuvo presente en la apertura de...,http://www.clarin.com/deportes/polideportivo/j...,0
138,138,1. El televisor más real que la realidad mi...,08/10/2017,//images.clarin.com/2017/10/05/r18QKz4h-_930x5...,"Lo que anticipó la Feria IFA. Berlín fue, una ...",/viva/,Los mejores y los peores electrodomésticos par...,http://www.clarin.com/viva/mejores-peores-elec...,0
130,130,"Las jubilaciones, el salario mínimo y la Asign...",10 de octubre de 2017,https://images.pagina12.com.ar/styles/width960...,NaN,Economía,Ingresos devaluados,https://www.pagina12.com.ar/68135-ingresos-dev...,1
72,72,El Maestro que apenas horas atrás nos enseña...,05/10/2017,//images.clarin.com/2017/10/05/rJGChVN3b_930x5...,OTRAS CARTAS. El relato del Partido Comunista ...,/cartas_al_pais/,"Edgardo, gruñón querido, la gente te sentía co...",http://www.clarin.com/cartas-al-pais/edgardo-g...,0
257,257,El seleccionado argentino femenino de hocke...,23/07/2017,NaN,Argentina perdió ante Inglaterra y finalizó en...,/deportes/hockey/,Las Leonas se despidieron de Sudáfrica con una...,http://www.clarin.com/deportes/hockey/leonas-d...,0
144,144,Familiares y víctimas de abusos sexuales en el...,10 de octubre de 2017,https://images.pagina12.com.ar/styles/width960...,NaN,Sociedad,Padres del Próvolo,https://www.pagina12.com.ar/68156-padres-del-p...,1


### 1. Limpieza

#### 1.1 Faltantes

A partir del dataset observamos que los campos que probablemente contengan el vocabulario relevante son "cuerpo", "título" y "resumen".
Sacar del análisis los registros que no tienen cuerpo o título disponible y completar los resúmenes faltantes con una campo en blanco


In [6]:
df.isnull().sum()

Unnamed: 0      0
cuerpo          6
fecha_hora      3
imagen         22
resumen       128
suplemento     68
titulo          3
url             0
class           0
dtype: int64

In [7]:
df=df[df['cuerpo'].notnull()]
df=df[df['resumen'].notnull()]
df=df[df['titulo'].notnull()]

In [8]:
df.isnull().sum()

Unnamed: 0     0
cuerpo         0
fecha_hora     0
imagen         9
resumen        0
suplemento    22
titulo         0
url            0
class          0
dtype: int64

In [9]:
df.drop(columns='imagen',inplace=True)
df.drop(columns='url',inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)

In [10]:
df.sample(10)

,cuerpo,fecha_hora,resumen,suplemento,titulo,class
105,Con escasísimo margen de error. Así entrará ma...,09 de octubre de 2017,El director técnico argentino introduciría cam...,Deportes,Las dudas de Sampaoli para enfrentar a Ecuador,1
60,PáginaI12 En España\nDesde Barcelona \nSi el d...,09 de octubre de 2017,La manifestación multitudinaria de ayer y la f...,El mundo,Nuevos palos en la rueda independentista catalana,1
74,"La prórroga de Ley 26.160, que es presentada...",03/10/2017,OTRAS CARTAS. Sentido adiós de televidentes a ...,/cartas_al_pais/,Sembraron la semilla del conflicto continuo en...,0
344,Pese a la gran cantidad de portales de e-co...,09/10/2017,Used es un catálogo online para que los usuari...,/autos/,Claas presenta su sistema de venta de usados,0
244,"Hijo de un padre maltratador, ladrón de auto...",21/09/2017,Murió uno de los más polémicos y fascinantes d...,/deportes/boxeo/,LaMotta y una vida de película que mutó del bo...,0
5,"En un nuevo acto de campaña, esta vez en Malv...",08 de octubre de 2017,La ex presidenta y candidata a senadora encabe...,El país,“Están a un pasito de volver a hablar de campa...,1
0,"El Ministerio de Agroindustria de la Nación,...",10/10/2017,De acuerdo a los datos relevados por el Instit...,/lo-ultimo/,Soja: aumentó la superficie sembrada con semil...,0
86,Los dos principales candidatos a gobernador ...,08/10/2017,Son comicios para gobernador. Tanto en el ofic...,/cultura/,Corrientes: ya votaron los candidatos y hay mu...,0
44,La Justicia Federal de Rosario procesó a Ari...,10/10/2017,"Ariel ""Guille"" Cantero está detenido en el pen...",/policiales/,"Procesan a líder de ""Los Monos"" por planificar...",0
162,"Son más de 35 chicos y jóvenes, de ambos sex...",28/09/2017,En el club de Villa Crespo funciona una escuel...,/deportes/ascenso/,Querido Ascenso: En Atlanta saben tirar parede...,0


In [11]:
df.suplemento.unique()

array(['/lo-ultimo/', '/politica/', '/mundo/', '/sociedad/',
       '/policiales/', '/ciudades/', '/opinion/', '/cartas_al_pais/',
       '/cultura/', '/rural/', '/economia/', '/tecnologia/',
       '/revista-enie/', '/viva/', '/br/', '/deportes/futbol/',
       '/deportes/ascenso/', '/deportes/futbol-internacional/',
       '/deportes/messi/', '/deportes/rugby/', '/deportes/tenis/',
       '/deportes/automovilismo/', '/deportes/basquet/',
       '/deportes/polideportivo/', '/deportes/turf/', '/deportes/boxeo/',
       '/deportes/hockey/', '/deportes/', '/espectaculos/fama/',
       '/espectaculos/tv/', '/espectaculos/cine/',
       '/espectaculos/musica/', '/espectaculos/teatro/', '/si/',
       '/espectaculos/', '/entremujeres/', '/autos/', '/buena-vida/',
       '/viajes/', '/arq/', 'El país', 'El mundo', 'Sociedad', 'Deportes',
       nan, 'Economía', 'Universidad', 'Plástica'], dtype=object)

In [12]:
import re
patron_deportes=r'(deportes)'
df['suplemento deportes']=df['suplemento'].str.extract(patron_deportes, flags=re.IGNORECASE)


In [13]:
df['suplemento deportes'].unique()

array([nan, 'deportes', 'Deportes'], dtype=object)

In [14]:
df=df[(df['suplemento deportes'].isnull())]
df.drop(columns='suplemento deportes',inplace=True)

In [15]:
df.suplemento.unique()

array(['/lo-ultimo/', '/politica/', '/mundo/', '/sociedad/',
       '/policiales/', '/ciudades/', '/opinion/', '/cartas_al_pais/',
       '/cultura/', '/rural/', '/economia/', '/tecnologia/',
       '/revista-enie/', '/viva/', '/br/', '/espectaculos/fama/',
       '/espectaculos/tv/', '/espectaculos/cine/',
       '/espectaculos/musica/', '/espectaculos/teatro/', '/si/',
       '/espectaculos/', '/entremujeres/', '/autos/', '/buena-vida/',
       '/viajes/', '/arq/', 'El país', 'El mundo', 'Sociedad', nan,
       'Economía', 'Universidad', 'Plástica'], dtype=object)

In [16]:
df

,cuerpo,fecha_hora,resumen,suplemento,titulo,class
0,"El Ministerio de Agroindustria de la Nación,...",10/10/2017,De acuerdo a los datos relevados por el Instit...,/lo-ultimo/,Soja: aumentó la superficie sembrada con semil...,0
1,"Políticos, economistas, funcionarios, empres...",10/10/2017,Los sondeos para las próximas elecciones y la ...,/lo-ultimo/,"Políticos, empresarios y periodistas en la cel...",0
2,En el Mercado de Liniers ingresaron 7.406 an...,10/10/2017,Las ventas resultaron ágiles y los mejores lot...,/lo-ultimo/,Mercado de Liniers: entrada pobre y recuperaci...,0
3,Cuando hace más de un año se supo del encuen...,10/10/2017,La actriz filmó una serie para Netflix en la q...,/lo-ultimo/,Kate del Castillo cuenta su versión del encuen...,0
4,En la pelea contra el efectivo y a favor del...,10/10/2017,El sistema de pago PIM que desarrolló el Banco...,/lo-ultimo/,La billetera móvil del Banco Nación llega a lo...,0
...,...,...,...,...,...,...
215,"Moira Millan, dirigente mapuche de la comunida...",10 de octubre de 2017,"La referente de la comunidad Pillán Mahuiza, d...",El país,Dirigente mapuche denuncia amenazas,1
216,El ex ministro de Transporte y candidato a sen...,10 de octubre de 2017,El candidato a senador por Cumplir se distanci...,El país,"""Tengo diferencias de fondo con este Gobierno""",1
217,En la conferencia de prensa brindada hoy por l...,10 de octubre de 2017,La ex presidenta fue crítica con el gobierno d...,El país,"""España está ante una encrucijada""",1
219,"""Asumo el mandato del puebl...",10 de octubre de 2017,"El presidente de la Generalitat, Carles Puigde...",El mundo,"""No somos locos ni golpistas""",1


#### 1.2 Suplementos relevantes

Para mejorar la clasificación es conveniente retirar las secciones donde los dos diarios utilizan un vocabulario similar y muy específico del dominio como, por ejemplo, las relacionadas a deportes.

#### 1.3 Corpus

Construir la columna sobre la cual vamos a predecir concatenando el título, resumen y cuerpo de las distintas noticias.

In [17]:
corpus= df[['titulo','resumen','cuerpo','class']]


In [19]:
corpus['noticia'] = (corpus['titulo'] + ' ' + corpus['cuerpo'])


In [20]:
corpus

,titulo,resumen,cuerpo,class,noticia
0,Soja: aumentó la superficie sembrada con semil...,De acuerdo a los datos relevados por el Instit...,"El Ministerio de Agroindustria de la Nación,...",0,Soja: aumentó la superficie sembrada con semil...
1,"Políticos, empresarios y periodistas en la cel...",Los sondeos para las próximas elecciones y la ...,"Políticos, economistas, funcionarios, empres...",0,"Políticos, empresarios y periodistas en la cel..."
2,Mercado de Liniers: entrada pobre y recuperaci...,Las ventas resultaron ágiles y los mejores lot...,En el Mercado de Liniers ingresaron 7.406 an...,0,Mercado de Liniers: entrada pobre y recuperaci...
3,Kate del Castillo cuenta su versión del encuen...,La actriz filmó una serie para Netflix en la q...,Cuando hace más de un año se supo del encuen...,0,Kate del Castillo cuenta su versión del encuen...
4,La billetera móvil del Banco Nación llega a lo...,El sistema de pago PIM que desarrolló el Banco...,En la pelea contra el efectivo y a favor del...,0,La billetera móvil del Banco Nación llega a lo...
...,...,...,...,...,...
215,Dirigente mapuche denuncia amenazas,"La referente de la comunidad Pillán Mahuiza, d...","Moira Millan, dirigente mapuche de la comunida...",1,Dirigente mapuche denuncia amenazas Moira Mill...
216,"""Tengo diferencias de fondo con este Gobierno""",El candidato a senador por Cumplir se distanci...,El ex ministro de Transporte y candidato a sen...,1,"""Tengo diferencias de fondo con este Gobierno""..."
217,"""España está ante una encrucijada""",La ex presidenta fue crítica con el gobierno d...,En la conferencia de prensa brindada hoy por l...,1,"""España está ante una encrucijada"" En la confe..."
219,"""No somos locos ni golpistas""","El presidente de la Generalitat, Carles Puigde...","""Asumo el mandato del puebl...",1,"""No somos locos ni golpistas"" ..."


#### 1.4 Normalización del texto

Primero es conveniente pasar todo el corpus a letras minúsculas.

Para mejorar el clasificador es importante sacar todos los acentos que pueden generar diferencias artificiales entre palabras según estén o no puestos. Hint: Utilizar el módulo unidecode

Una vez retirados los acentos, quitar todos los signos de puntuación para dejar únicamente palabras. Hint: utilizar una expresión regular, por ejemplo, r'([^\s\w]|_)+' para reemplazar todo lo que no sean palabras.

In [21]:
# Normalizamos el texto
corpus['noticia'] = corpus['noticia'].apply(lambda x: unicodedata.normalize("NFKD", x.lower()))
corpus.head(

,titulo,resumen,cuerpo,class,noticia
0,Soja: aumentó la superficie sembrada con semil...,De acuerdo a los datos relevados por el Instit...,"El Ministerio de Agroindustria de la Nación,...",0,soja: aumentó la superficie sembrada con semi...
1,"Políticos, empresarios y periodistas en la cel...",Los sondeos para las próximas elecciones y la ...,"Políticos, economistas, funcionarios, empres...",0,"políticos, empresarios y periodistas en la ce..."
2,Mercado de Liniers: entrada pobre y recuperaci...,Las ventas resultaron ágiles y los mejores lot...,En el Mercado de Liniers ingresaron 7.406 an...,0,mercado de liniers: entrada pobre y recuperaci...
3,Kate del Castillo cuenta su versión del encuen...,La actriz filmó una serie para Netflix en la q...,Cuando hace más de un año se supo del encuen...,0,kate del castillo cuenta su versión del encue...
4,La billetera móvil del Banco Nación llega a lo...,El sistema de pago PIM que desarrolló el Banco...,En la pelea contra el efectivo y a favor del...,0,la billetera móvil del banco nación llega a ...


In [32]:
#patten_words=r'([^\s\w]|_)+'
#corpus['noticia_regex']=corpus['noticia'].str.extract(patten_words, flags=re.IGNORECASE)
from nltk.stem.snowball import SnowballStemmer


In [33]:
def stemfraseesp(frase):
    
    token_words=word_tokenize(frase)
    token_words
    stem_sentence=[]
    
    spanishStemmer=SnowballStemmer("spanish")
    
    for word in token_words:
        stem_sentence.append(spanishStemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [34]:
stemfraseesp(corpus.iloc[0]['noticia'])

'soj : aumentó la superfici sembr con semill fiscaliz el ministeri de agroindustri de la nación , a través del institut nacional de semill ( inas ) , publicó un inform acerc de la identificación de usuari , variedad , superfici de siembr y uso de semill . en el document , elabor por el registr de usuari de semill ( rus ) , se desprend que la superfici sembr con soj que fue de 18 millon de hectár par la últim campañ , la información de las declar jur registr alcanz el 36,3 % de la superfici sembr en tod el país , según un comunic de la carter agropecuari . mirá también baj el escrutini del públic al compar las cifr con la campañ anterior , sig el comunic , se observ un increment del 4,25 % en la superfici declar , en tant que el volum de semill sembr aumentó un 12,52 % . entre ambas campañ , la cantid de productor registr creció un 14,95 % en referent a los productor inclu en la categorí de grand usuari de semill ( gus ) . si bien la evident del aument es signific , s

In [49]:
corpus['noticia']=corpus['noticia'].apply(lambda x:(remove_punctuation(pattern,x)))
corpus['noticia stem'] = corpus['noticia'].apply(stemfraseesp)

TypeError: expected string or bytes-like object

In [36]:
corpus

,titulo,resumen,cuerpo,class,noticia,noticia stem
0,Soja: aumentó la superficie sembrada con semil...,De acuerdo a los datos relevados por el Instit...,"El Ministerio de Agroindustria de la Nación,...",0,soja: aumentó la superficie sembrada con semi...,soj : aumentó la superfici sembr con semill f...
1,"Políticos, empresarios y periodistas en la cel...",Los sondeos para las próximas elecciones y la ...,"Políticos, economistas, funcionarios, empres...",0,"políticos, empresarios y periodistas en la ce...","polít , empresari y period en la celebración..."
2,Mercado de Liniers: entrada pobre y recuperaci...,Las ventas resultaron ágiles y los mejores lot...,En el Mercado de Liniers ingresaron 7.406 an...,0,mercado de liniers: entrada pobre y recuperaci...,merc de liniers : entrad pobr y recuperación ...
3,Kate del Castillo cuenta su versión del encuen...,La actriz filmó una serie para Netflix en la q...,Cuando hace más de un año se supo del encuen...,0,kate del castillo cuenta su versión del encue...,kat del castill cuent su versión del encuentr...
4,La billetera móvil del Banco Nación llega a lo...,El sistema de pago PIM que desarrolló el Banco...,En la pelea contra el efectivo y a favor del...,0,la billetera móvil del banco nación llega a ...,la billeter móvil del banc nación lleg a los...
...,...,...,...,...,...,...
215,Dirigente mapuche denuncia amenazas,"La referente de la comunidad Pillán Mahuiza, d...","Moira Millan, dirigente mapuche de la comunida...",1,dirigente mapuche denuncia amenazas moira mill...,"dirigent mapuch denunci amenaz moir mill , dir..."
216,"""Tengo diferencias de fondo con este Gobierno""",El candidato a senador por Cumplir se distanci...,El ex ministro de Transporte y candidato a sen...,1,"""tengo diferencias de fondo con este gobierno""...",`` teng diferent de fond con este gobiern '' e...
217,"""España está ante una encrucijada""",La ex presidenta fue crítica con el gobierno d...,En la conferencia de prensa brindada hoy por l...,1,"""españa está ante una encrucijada"" en la con...",`` españ está ante una encrucij '' en la con...
219,"""No somos locos ni golpistas""","El presidente de la Generalitat, Carles Puigde...","""Asumo el mandato del puebl...",1,"""no somos locos ni golpistas"" ...",`` no som loc ni golpist '' `` asum el mandat ...


## 2. Modelo

Vectorizar el corpus resultante con TF-IDF y aplicar un modelo Naive Bayes con un split simple entre train y test. 
<br />
¿Cuál es la dimensión de la matriz de features? ¿Cuál es el accuracy obtenido? <br />
Dibujar la matriz de confusión.

In [37]:
X=corpus['noticia stem']
y=corpus['class']

In [38]:
spanish_stop_words=pd.read_csv('lista de stopwords.csv',encoding='latin1')

In [60]:
spanish_stop_words

,a
0,a
1,acá
2,ademas
3,además
4,ahí
...,...
207,verdadera
208,voy
209,y
210,ya


In [73]:
spanish_stopwords = stopwords.words('spanish')
len(spanish_stopwords)

313

In [42]:
y.value_counts(normalize=True)

0    0.726225
1    0.273775
Name: class, dtype: float64

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                    random_state=42,
                                                   stratify=y)


In [44]:
X_train

151    “ no hay iguald sin cambi ” vari especial en g...
295    un thrill imprescind tal vez dav mackenzi ni s...
341    de buen air a nuev york : viaj a las entrañ d...
125    la muj conduc el cambi en arabi arabi saudit h...
293    murió el actor jean rochefort , reconoc figur...
                             ...                        
94     la cám de puert priv pidió transparent los c...
327    martín bossi se disfrazó de lionel messi y l...
88     robert cap : el fotógraf de la guerr , en tie...
22     el president catalán carl puigdemont declaró...
37     avanz vari proyect de ley par pen la tenenci d...
Name: noticia stem, Length: 277, dtype: object

In [55]:
from sklearn.preprocessing import FunctionTransformer


ModuleNotFoundError: No module named 'unicode'

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(analyzer='word')),
    ('dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
    ('classifier',None)])

parameters =[
        
        dict(dict(vect__stop_words=[spanish_stopwords,None],
             vect__ngram_range=[(1,1),(1,2)],
             vect__min_df=[0]),
             classifier__n_estimators=[600],
             classifier__random_state=[seed],
             classifier__min_samples_split=[2,3],
             classifier__min_samples_leaf=[5],
             classifier__bootstrap=[True,False],
             classifier__max_depth=[3,4],
             classifier__class_weight=['balanced',None],
             classifier=[RandomForestClassifier()])]

gs = GridSearchCV(pipeline, parameters, cv=5,
                               n_jobs=-1, verbose=1,scoring='roc_auc',return_train_score=False).fit(X_train, y_train)
gs.fit(X_train, y_train).best_params_

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  5.3min finished


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.5min


dict(dict(#vect__stop_words=['Spanish',None],
             vect__ngram_range=[(1,1),(1,2)],
             vect__min_df=[2,5]),
            classifier__C=[1,1e20],
             classifier__solver=['lbfgs'],
             classifier__max_iter=[300,500],
             classifier__class_weight=['balanced',None],
             classifier=[LogisticRegression()]),
        dict(dict(#vect__stop_words=['Spanish',None],
             vect__ngram_range=[(1,1),(1,2)],
             vect__min_df=[2,5]),
            classifier=[BernoulliNB()]),
        dict(dict(#vect__stop_words=['Spanish',None],
             vect__ngram_range=[(1,1),(1,2)],
             vect__min_df=[2,5]),
            classifier=[GaussianNB()]),

In [114]:
gs.best_score_

0.9474036955433162

In [115]:
pred_tra=gs.predict(X_train)

In [116]:
print(classification_report(y_train,pred_tra))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       201
           1       1.00      0.97      0.99        76

    accuracy                           0.99       277
   macro avg       1.00      0.99      0.99       277
weighted avg       0.99      0.99      0.99       277



In [117]:
pred=gs.predict(X_test)

In [118]:
fpr, tpr, thresholds =roc_curve(y_test, pred)
auc(fpr, tpr)

0.7502579979360166

In [119]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89        51
           1       0.73      0.58      0.65        19

    accuracy                           0.83        70
   macro avg       0.79      0.75      0.77        70
weighted avg       0.82      0.83      0.82        70



In [120]:
clarin='Marcaron presencia los embajadores y representantes de las 22 sedes que los 28 miembros de la Unión Europea tienen en la Argentina. Alberto Fernández llegó pasadas las 13 horas y se fue poco antes de las 16. Era su primer encuentro como presidente electo con los jefes de las embajadas de la UE.Así, este jueves Fernandez se definió ante los diplomáticos de la UE en Argentina como un “europeista”, y ante ellos también pidió ayuda para que el país afronte sus problemas de deuda pendientes con el Fondo Monetario Internacional. Y les pidió que lleguen más inversiones de la región.Alberto Fernández afirmó que espera que el Gobierno argentino “no reconozca” la presidencia interina de Jeanine Áñez en BoliviaAdemás, lejos de las críticas que en la campaña electoral tuvo Alberto F. para con el acuerdo de libre comercio entre ese bloque y el Mercosur anunciado el 28, Fernández se mostró partidario de un asociación en ese sentido aunque dijo que lo firmado debía revisarlo porque no conocía su contenido.'


In [121]:
print(clarin)

Marcaron presencia los embajadores y representantes de las 22 sedes que los 28 miembros de la Unión Europea tienen en la Argentina. Alberto Fernández llegó pasadas las 13 horas y se fue poco antes de las 16. Era su primer encuentro como presidente electo con los jefes de las embajadas de la UE.Así, este jueves Fernandez se definió ante los diplomáticos de la UE en Argentina como un “europeista”, y ante ellos también pidió ayuda para que el país afronte sus problemas de deuda pendientes con el Fondo Monetario Internacional. Y les pidió que lleguen más inversiones de la región.Alberto Fernández afirmó que espera que el Gobierno argentino “no reconozca” la presidencia interina de Jeanine Áñez en BoliviaAdemás, lejos de las críticas que en la campaña electoral tuvo Alberto F. para con el acuerdo de libre comercio entre ese bloque y el Mercosur anunciado el 28, Fernández se mostró partidario de un asociación en ese sentido aunque dijo que lo firmado debía revisarlo porque no conocía su cont

In [122]:
pagina12='Por ahora, Cohen está procesado por falso testimonio agravado, que es una figura grave que puede implicar hasta diez años de cárcel. Pero Baratta pide que sea ampliada su indagatoria, ya que considera que su responsabilidad es mayor y abarca maniobras más complejas. La causa a cargo de Ramos y de la fiscal Paloma Ochoa, se focalizó hasta el momento en la responsabilidad de Cohen en la redacción de un peritaje que contenía tramos copiados de una tesis de dos estudiantes chilenos y que invocaba un parámetro para calcular costos que es un valor llamado Henry Hub, que es un precio estadounidense interno en boca de gasoducto. No es un precio internacional ni Estados Unidos exportó GNL en el tramo investigado. Pero Cohen con esa referencia hablaba del pago de millones de dólares de sobreprecios. Para colmo hizo un dictamen propio, separado del resto de los peritos, que fue el que tuvo en cuenta el juez Claudio Bonadío. La causa llevó a Baratta y a julio De Vido a la cárcel. Cuando se venía venir que era un estudio trucho, apareció el falso abogado Marcelo Sebastián D’Alessio en la causa. A la larga el juez cambió el eje y señaló otras irregularidades, no ya sobreprecios.'

In [123]:
 print(pagina12)

Por ahora, Cohen está procesado por falso testimonio agravado, que es una figura grave que puede implicar hasta diez años de cárcel. Pero Baratta pide que sea ampliada su indagatoria, ya que considera que su responsabilidad es mayor y abarca maniobras más complejas. La causa a cargo de Ramos y de la fiscal Paloma Ochoa, se focalizó hasta el momento en la responsabilidad de Cohen en la redacción de un peritaje que contenía tramos copiados de una tesis de dos estudiantes chilenos y que invocaba un parámetro para calcular costos que es un valor llamado Henry Hub, que es un precio estadounidense interno en boca de gasoducto. No es un precio internacional ni Estados Unidos exportó GNL en el tramo investigado. Pero Cohen con esa referencia hablaba del pago de millones de dólares de sobreprecios. Para colmo hizo un dictamen propio, separado del resto de los peritos, que fue el que tuvo en cuenta el juez Claudio Bonadío. La causa llevó a Baratta y a julio De Vido a la cárcel. Cuando se venía v

In [124]:
print(gs.predict([clarin,pagina12]))


[0 0]
